<h1>Imports</h1>

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report

from imblearn.combine import SMOTETomek


Using TensorFlow backend.


In [2]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [4]:
DATA_PATH = Path.cwd() / "../../datasets"

In [5]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [6]:
training_df = training_df.drop('pickup_time', 1)
training_df = training_df.drop('drop_time', 1)

<h1>Data Preprocessing</h1>

In [7]:
features = training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','fare']]
labels = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
labels = labels.replace({'label':mapping})

In [8]:
numerical_features = features.columns[features.dtypes != "object"].values

In [9]:
X_train, X_eval, y_train, y_eval = train_test_split(features, labels, test_size=0.3, shuffle=True, stratify= labels, random_state=RANDOM_SEED)

<h1>Data Cleaning</h1>

<h1>Feature Engineering</h1>

<h1>Model Training</h1>

In [10]:

## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median')),
    ('minmax_scaler', MinMaxScaler())
])


## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [11]:
preprocessed_features = preprocessor.fit_transform(X_train)
preprocessed_features_data_frame = pd.DataFrame(data=preprocessed_features, columns=features.columns)

In [12]:
preprocessed_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_features, columns=features.columns)

In [13]:
smk = SMOTETomek(random_state=42)
os_x_train, os_y_train = smk.fit_sample(preprocessed_features_data_frame,y_train)

In [14]:
os_y_train.shape

(21372, 1)

In [15]:
estimators = MultiOutputClassifier(
    estimator = LogisticRegression(penalty='l2', C=1)
)

In [16]:
full_pipeline = Pipeline([
    ('estimators', estimators)
])

full_pipeline

Pipeline(memory=None,
         steps=[('estimators',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1,
                                                                    class_weight=None,
                                                                    dual=False,
                                                                    fit_intercept=True,
                                                                    intercept_scaling=1,
                                                                    l1_ratio=None,
                                                                    max_iter=100,
                                                                    multi_class='auto',
                                                                    n_jobs=None,
                                                                    penalty='l2',
                                                                    random_state=None,
                           

In [17]:


## Train the model
full_pipeline.fit(os_x_train, os_y_train)

# Predict for the evaluation set
print("Training Accuracy: %.2f" % (full_pipeline.score(preprocessed_test_features_data_frame, y_eval)*100), "%")
y_pred = full_pipeline.predict(preprocessed_test_features_data_frame)

Training Accuracy: 34.37 %


In [18]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

array([[ 431,   73],
       [3309, 1340]])

In [20]:
print(classification_report(y_eval, y_pred))

              precision    recall  f1-score   support

           0       0.12      0.86      0.20       504
           1       0.95      0.29      0.44      4649

    accuracy                           0.34      5153
   macro avg       0.53      0.57      0.32      5153
weighted avg       0.87      0.34      0.42      5153



In [ ]:
full_pipeline.fit(features,labels)

<h1>Model Validation</h1>

In [ ]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

In [ ]:
test_probs = full_pipeline.predict(test_set)

In [ ]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= test_probs

In [ ]:
submission_set.to_csv('../../submissions/model2/teamCluster_submission_01.csv', index=True)
print("Completed!")

In [ ]:
submission_set['prediction'].idxmin()

In [ ]:
submission_set['prediction'].value_counts()